In [1]:
!pip install --upgrade cerebras_cloud_sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 2.9 MB/s eta 0:00:00


In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
cerebras_api_key = user_secrets.get_secret("CEREBRAS_API")
gemini_api_key = user_secrets.get_secret("GOOGLE_API_KEY")
hf_api_key = user_secrets.get_secret("HF_API_KEY")

In [3]:
import os
import time
from cerebras.cloud.sdk import Cerebras

client = Cerebras(api_key=cerebras_api_key)

# Function to call GPT-OSS for text completion
def gpt_oss(prompt, 
            model="gpt-oss-120b", 
            temperature=0.0, 
            max_tokens=1024,
            verbose=False,
            max_tries=3):
    
    # Show details if verbose mode is on
    if verbose:
        print(f"Prompt:\n{prompt}\n")
        print(f"Model: {model}")
        print(f"Temperature: {temperature}")
        print(f"Max Tokens: {max_tokens}")

    # Structure the message for the API
    messages = [
        {
            "role": "user",
            "content": prompt,
        }
    ]
    
    # Allow multiple attempts to call the API in case of downtime or issues
    for num_tries in range(max_tries):
        try:
            # Make the API call to Cerebras GPT-OSS model
            chat_completion = client.chat.completions.create(
                messages=messages,
                model=model,
                temperature=temperature,
                max_tokens=max_tokens
            )
            
            # Print the full response if verbose
            if verbose:
                print(f"Full Response Object: {chat_completion}")
            
            # Check if the response contains choices and return the text
            if hasattr(chat_completion, 'choices') and len(chat_completion.choices) > 0:
                response_content = chat_completion.choices[0].message.content
                
                # Return the formatted response in a structured manner
                formatted_response = {
                    "model": model,
                    "prompt": prompt,
                    "response": response_content,
                    "temperature": temperature,
                    "max_tokens": max_tokens,
                    "status": "success"
                }
                
                return formatted_response

            # Handle unexpected response structure
            print("Unexpected response structure:", chat_completion)
            return None

        except Exception as e:
            print(f"Error: {e}")
            print(f"Attempt {num_tries + 1}/{max_tries}")
            
            # Simple retry logic with exponential backoff
            wait_time = 2 ** num_tries
            print(f"Waiting for {wait_time} seconds before retrying...")
            time.sleep(wait_time)
    
    # Return failure if we exhaust the retries
    print(f"Tried {max_tries} times, but failed to get a valid response.")
    return {
        "model": model,
        "prompt": prompt,
        "response": None,
        "status": "failure",
        "error": "Exceeded max retries"
    }

# Function to handle chat-like interactions (prompt-response pairs)
def gpt_oss_chat(prompts, responses,
                 model="gpt-oss-120b", 
                 temperature=0.0, 
                 max_tokens=1024,
                 verbose=False,
                 max_tries=3):
    
    # Generate the prompt for chat interactions
    prompt = get_prompt_chat(prompts, responses)

    # Call gpt_oss with the generated chat prompt
    return gpt_oss(prompt, model=model, temperature=temperature, 
                   max_tokens=max_tokens, verbose=verbose, max_tries=max_tries)

# Function to build the prompt for chat interactions
def get_prompt_chat(prompts, responses):
    prompt_chat = f"<s>[INST] {prompts[0]} [/INST]"
    for n, response in enumerate(responses):
        prompt = prompts[n + 1]
        prompt_chat += f"\n{response}\n</s><s>[INST] \n{prompt}\n [/INST]"
    return prompt_chat

### LLMs are Stateless

In [5]:
prompt = """
    What are fun activities I can do this weekend?
"""
response = gpt_oss(prompt)
print(response)

{'model': 'gpt-oss-120b', 'prompt': '\n    What are fun activities I can do this weekend?\n', 'response': 'Sure! I can suggest a mix of ideas, but it’ll be even better if I know a little more about your situation. Could you let me know:\n\n1. **Where you’ll be** (city, town, or general region) – so I can suggest local spots or events.  \n2. **Who you’ll be with** (alone, friends, family, a partner, kids, a pet, etc.).  \n3. **Your interests** (outdoors, arts, food, sports, games, learning something new, relaxing, etc.).  \n4. **Budget** (free, low‑cost, or you’re happy to splurge).  \n5. **Any weather constraints** (e.g., it’s supposed to rain, or you love sunny days).  \n\nOnce I have a bit more detail, I’ll tailor a list of fun activities that fit your weekend perfectly!', 'temperature': 0.0, 'max_tokens': 1024, 'status': 'success'}


In [6]:
prompt_2 = """
Which of these would be good for my health?
"""
response_2 = gpt_oss(prompt_2)
print(response_2)

{'model': 'gpt-oss-120b', 'prompt': '\nWhich of these would be good for my health?\n', 'response': 'I’m happy to help you figure out what’s good for your health, but I need a little more information first. Could you let me know what “these” refers to? For example:\n\n- A list of foods or drinks you’re considering (e.g., different fruits, snacks, beverages)  \n- A set of exercise options or fitness classes  \n- Various lifestyle habits (e.g., sleep patterns, screen time, stress‑relief techniques)  \n- Specific products or supplements you’re curious about  \n\nOnce I know what items you’re comparing, I can give you a clear, evidence‑based rundown of the pros and cons for each and suggest which ones are most likely to support your overall well‑being. Feel free to paste the list or describe the choices, and we’ll dive right in!', 'temperature': 0.0, 'max_tokens': 1024, 'status': 'success'}


### Constructing multi-turn prompts
You need to provide prior prompts and responses as part of the context of each new turn in the conversation.

In [7]:
prompt_1 = """
    What are fun activities I can do this weekend?
"""
response_1 = gpt_oss(prompt_1)

In [8]:
prompt_2 = """
Which of these would be good for my health?
"""

In [9]:
chat_prompt = f"""
<s>[INST] {prompt_1} [/INST]
{response_1}
</s>
<s>[INST] {prompt_2} [/INST]
"""
print(chat_prompt)


<s>[INST] 
    What are fun activities I can do this weekend?
 [/INST]
{'model': 'gpt-oss-120b', 'prompt': '\n    What are fun activities I can do this weekend?\n', 'response': 'Sure! I can suggest a mix of ideas, but it’ll be even better if I know a little more about your situation. Could you let me know:\n\n1. **Where you’ll be** (city, town, or general region) – so I can suggest local spots or events.  \n2. **Who you’ll be with** (alone, friends, family, a partner, kids, a pet, etc.).  \n3. **Your interests** (outdoors, arts, food, sports, games, learning something new, relaxing, etc.).  \n4. **Budget** (free, low‑cost, or you’re happy to splurge).  \n5. **Any weather constraints** (e.g., it’s supposed to rain, or you love sunny days).  \n\nOnce I have a bit more detail, I’ll tailor a list of fun activities that fit your weekend perfectly!', 'temperature': 0.0, 'max_tokens': 1024, 'status': 'success'}
</s>
<s>[INST] 
Which of these would be good for my health?
 [/INST]



In [13]:
response_2 = gpt_oss(chat_prompt,
                     verbose=False)
print(response_2)

{'model': 'gpt-oss-120b', 'prompt': "\n<s>[INST] \n    What are fun activities I can do this weekend?\n [/INST]\n{'model': 'gpt-oss-120b', 'prompt': '\\n    What are fun activities I can do this weekend?\\n', 'response': 'Sure! I can suggest a mix of ideas, but it’ll be even better if I know a little more about your situation. Could you let me know:\\n\\n1. **Where you’ll be** (city, town, or general region) – so I can suggest local spots or events.  \\n2. **Who you’ll be with** (alone, friends, family, a partner, kids, a pet, etc.).  \\n3. **Your interests** (outdoors, arts, food, sports, games, learning something new, relaxing, etc.).  \\n4. **Budget** (free, low‑cost, or you’re happy to splurge).  \\n5. **Any weather constraints** (e.g., it’s supposed to rain, or you love sunny days).  \\n\\nOnce I have a bit more detail, I’ll tailor a list of fun activities that fit your weekend perfectly!', 'temperature': 0.0, 'max_tokens': 1024, 'status': 'success'}\n</s>\n<s>[INST] \nWhich of th

### Use GPT-OSS chat helper function

In [14]:
prompt_1 = """
    What are fun activities I can do this weekend?
"""
response_1 = gpt_oss(prompt_1)

In [15]:
prompt_2 = """
Which of these would be good for my health?
"""

In [16]:
prompts = [prompt_1,prompt_2]
responses = [response_1]

In [18]:
# Pass prompts and responses to llama_chat function
response_2 = gpt_oss_chat(prompts,responses,verbose=True)

Prompt:
<s>[INST] 
    What are fun activities I can do this weekend?
 [/INST]
{'model': 'gpt-oss-120b', 'prompt': '\n    What are fun activities I can do this weekend?\n', 'response': 'Sure! I can suggest a mix of ideas, but it’ll be even better if I know a little more about your situation. Could you let me know:\n\n1. **Where you’ll be** (city, town, or general region) – so I can suggest local spots or events.  \n2. **Who you’ll be with** (alone, friends, family, a partner, kids, a pet, etc.).  \n3. **Your interests** (outdoors, arts, food, sports, games, learning something new, relaxing, etc.).  \n4. **Budget** (free, low‑cost, or you’re happy to splurge).  \n5. **Any weather constraints** (e.g., it’s supposed to rain, or you love sunny days).  \n\nOnce I have a bit more detail, I’ll tailor a list of fun activities that fit your weekend perfectly!', 'temperature': 0.0, 'max_tokens': 1024, 'status': 'success'}
</s><s>[INST] 

Which of these would be good for my health?

 [/INST]

Mod

In [19]:
print(response_2)

{'model': 'gpt-oss-120b', 'prompt': "<s>[INST] \n    What are fun activities I can do this weekend?\n [/INST]\n{'model': 'gpt-oss-120b', 'prompt': '\\n    What are fun activities I can do this weekend?\\n', 'response': 'Sure! I can suggest a mix of ideas, but it’ll be even better if I know a little more about your situation. Could you let me know:\\n\\n1. **Where you’ll be** (city, town, or general region) – so I can suggest local spots or events.  \\n2. **Who you’ll be with** (alone, friends, family, a partner, kids, a pet, etc.).  \\n3. **Your interests** (outdoors, arts, food, sports, games, learning something new, relaxing, etc.).  \\n4. **Budget** (free, low‑cost, or you’re happy to splurge).  \\n5. **Any weather constraints** (e.g., it’s supposed to rain, or you love sunny days).  \\n\\nOnce I have a bit more detail, I’ll tailor a list of fun activities that fit your weekend perfectly!', 'temperature': 0.0, 'max_tokens': 1024, 'status': 'success'}\n</s><s>[INST] \n\nWhich of thes

In [22]:
# replace prompt_3 with your own question!
prompt_3 = "Which of these activites would be fun with friends?"
prompts = [prompt_1, prompt_2, prompt_3]
responses = [response_1, response_2]

response_3 = gpt_oss_chat(prompts, responses, verbose=True)

print(response_3)

Prompt:
<s>[INST] 
    What are fun activities I can do this weekend?
 [/INST]
{'model': 'gpt-oss-120b', 'prompt': '\n    What are fun activities I can do this weekend?\n', 'response': 'Sure! I can suggest a mix of ideas, but it’ll be even better if I know a little more about your situation. Could you let me know:\n\n1. **Where you’ll be** (city, town, or general region) – so I can suggest local spots or events.  \n2. **Who you’ll be with** (alone, friends, family, a partner, kids, a pet, etc.).  \n3. **Your interests** (outdoors, arts, food, sports, games, learning something new, relaxing, etc.).  \n4. **Budget** (free, low‑cost, or you’re happy to splurge).  \n5. **Any weather constraints** (e.g., it’s supposed to rain, or you love sunny days).  \n\nOnce I have a bit more detail, I’ll tailor a list of fun activities that fit your weekend perfectly!', 'temperature': 0.0, 'max_tokens': 1024, 'status': 'success'}
</s><s>[INST] 

Which of these would be good for my health?

 [/INST]
{'mo